# Data Processing: Subset Selection for Dataset Diversity

This notebook demonstrates how to use **subset selection** techniques to identify representative samples from large datasets. The implementation uses facility location maximization with embedding-based similarity to select diverse subsets that cover the full distribution of the data.

## Use Cases

Subset selection is particularly useful for:
- Reducing dataset size while maintaining diversity
- Selecting training data that covers the full distribution
- Creating validation/test sets that represent the full dataset
- Identifying core samples for data annotation or review

## Requirements

**Note:** This notebook requires GPU acceleration for optimal performance.

## 📦 Installation

Install the required packages for subset selection. Run this once per session. If you restart the kernel or change runtimes, re-run this cell before continuing.

In [ ]:
# Install subset selection dependencies
# See scripts/subset_selection/requirements.txt for the full list
%pip install -qq -r ../../scripts/subset_selection/requirements.txt

# Note: This installs:
# - torch, transformers, numpy (ML & embeddings)
# - datasets, h5py (data processing)
# - submodlib-py (subset selection optimization)
# - jinja2, tqdm (templating & progress bars)

## 🔧 Configuration

### Import Required Modules

First, let's import the subset selection functionality from the scripts package.


In [ ]:
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

# Import subset selection modules
from scripts.subset_selection import (
    subset_datasets,
    BasicConfig,
    EncoderConfig,
    TemplateConfig,
    SystemConfig,
    get_supported_encoders
)

print("✓ Successfully imported subset selection modules")
print(f"✓ Project root: {project_root}")
print(f"✓ Supported encoders: {get_supported_encoders()}")


### Configure Parameters

Configure all the key parameters for subset selection.

**Input Files**: Supported formats include JSONL, JSON, CSV, and Parquet  
**Subset Sizes**: Use fractions (0.1 = 10%) or absolute counts (1000 = exactly 1000 samples)  
**Output Directory**: Where to save results (embeddings, metadata, subset files)


In [ ]:
# ============================================================================
# INPUT CONFIGURATION
# ============================================================================
# Specify dataset file(s) to process (JSONL, JSON, CSV, or Parquet)
input_files = [
    # TODO: Replace with your actual dataset path
    # Example: "path/to/your/dataset.jsonl"
    "../assets/subset-selection/combined_cut_50x.jsonl"  # <-- REPLACE THIS PATH
]

# ============================================================================
# SUBSET CONFIGURATION
# ============================================================================
# Subset sizes: Use fractions (0.1 = 10%) or absolute counts (1000 = exactly 1000 samples)
subset_sizes = [0.1, 0.5]

# ============================================================================
# OUTPUT CONFIGURATION
# ============================================================================
# Directory to save results (embeddings, metadata, subset files)
output_dir = "subset-selection/output"

# ============================================================================
# BASIC CONFIGURATION
# ============================================================================
batch_size = 100000       # Number of samples to process at once
num_folds = 50           # Number of partitions for parallel processing
epsilon = 160.0          # Optimization quality vs speed (160.0 for large datasets, 0.1-1.0 for small)
combine_files = False    # Combine multiple input files before processing

# ============================================================================
# SYSTEM CONFIGURATION
# ============================================================================
num_gpus = None         # Number of GPUs (None = auto-detect)
seed = 42               # Random seed for reproducibility


### Display Configuration

Let's verify the configured parameters:

In [ ]:
from pathlib import Path

# Create output directory
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)

# Display configuration
print("="*80)
print("SUBSET SELECTION CONFIGURATION")
print("="*80)

print("\n📁 INPUT:")
print(f"  Files: {len(input_files)}")
for i, f in enumerate(input_files, 1):
    print(f"    {i}. {f}")

print(f"\n📊 SUBSET SIZES:")
print(f"  {subset_sizes}")

print(f"\n💾 OUTPUT:")
print(f"  Directory: {output_path.absolute()}")

print(f"\n⚙️  BASIC CONFIGURATION:")
print(f"  Batch size: {batch_size:,}")
print(f"  Number of folds: {num_folds}")
print(f"  Epsilon: {epsilon}")
print(f"  Combine files: {combine_files}")

print(f"\n🖥️  SYSTEM:")
print(f"  GPUs: {'auto-detect' if num_gpus is None else num_gpus}")
print(f"  Seed: {seed}")

print("\n" + "="*80)


### Advanced Configuration (Optional)

The following sections provide additional information about encoder and system configuration. These use defaults from the scripts and typically don't need to be changed.


In [ ]:
# Display encoder information from defaults
from scripts.subset_selection import EncoderConfig

encoder_config = EncoderConfig()

print("✓ Encoder configuration (using defaults from scripts):")
print(f"  - Encoder type: {encoder_config.encoder_type}")
print(f"  - Model: {encoder_config.encoder_model}")
print(f"  - Template: conversation")

## ✨ Run Subset Selection

Now we'll run the subset selection process with all the configurations set above.

The subset selection process includes:

1. **Data Loading**: Reading and preprocessing the input files
2. **Embedding Generation**: Computing embeddings for all samples using the Arctic encoder
3. **Subset Selection**: Using facility location optimization to select diverse subsets
4. **Output Generation**: Saving the selected subsets and metadata

**Note:** This may take a while depending on dataset size, number of subsets, and available GPU resources. Progress bars will show the status of each step.


In [ ]:
print("="*100)
print("Starting subset selection...")
print(f"  Input files: {input_files}")
print(f"  Subset sizes: {subset_sizes}")
print(f"  Output directory: {output_dir}")
print("="*100)

try:
    # Build kwargs dictionary with configured parameters
    kwargs = {
        "output_dir": output_dir,
        "batch_size": batch_size,
        "num_folds": num_folds,
        "epsilon": epsilon,
        "combine_files": combine_files,
        "seed": seed,
    }
    
    # Add optional parameters if specified
    if num_gpus is not None:
        kwargs["num_gpus"] = num_gpus
    
    # Run subset selection (encoder settings use defaults from scripts)
    subset_datasets(
        input_files=input_files,
        subset_sizes=subset_sizes,
        **kwargs
    )
    
    print("\n" + "="*100)
    print(f"✓ Subset selection complete!")
    print(f"✓ Results saved to: {Path(output_dir).absolute()}")
    print("="*100)
    
except Exception as e:
    print(f"\n✗ Error during subset selection: {e}")
    raise

## 🍩 Additional Resources

For more information about subset selection and data processing:

- **GitHub Repository**: [Open Data Hub Data Processing](https://github.com/opendatahub-io/odh-data-processing/)
- **Subset Selection README**: Check `scripts/subset_selection/README.md` for detailed documentation
- **API Documentation**: See `scripts/subset_selection/subset_selection.py` for advanced configuration options

### Technical Details

The subset selection algorithm uses:
- **Facility Location Maximization**: A submodular optimization technique for diverse subset selection
- **LazierThanLazyGreedy Optimizer**: An efficient greedy algorithm implementation
- **Embedding-based Similarity**: Uses neural embeddings to measure semantic similarity
- **Multi-GPU Parallelization**: Automatically distributes work across available GPUs

### Acknowledgement
This notebook and the subset selection implementation are based on the work from:

**DataCurate4LLMs** by [@krishnatejakk](https://github.com/krishnatejakk)  
Repository: https://github.com/krishnatejakk/DataCurate4LLMs

The original repository provides data curation techniques for language models, including subset selection using submodular optimization and embedding-based similarity. We've adapted and integrated their approach into the Open Data Hub - Data Processing toolkit.

### Any Feedback?

We'd love to hear if you have any feedback on this or any other notebook in this series! Please [open an issue](https://github.com/opendatahub-io/odh-data-processing/issues) and help us improve our demos.
